In [1]:
import numpy as np
import sklearn 
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from scipy.integrate import odeint
import math

# from utility_code.utility import utils

#import csvs
train = pd.read_csv("../../data/train.csv")
test = pd.read_csv("../../data/test.csv")
graph = pd.read_csv("../../data/graph.csv")

statesdata = {}
states = pd.Series.unique(train['Province_State'])
num_states = len(states)
for s in states:
    statesdata[s] = train.loc[train['Province_State'] == s ,:]

In [4]:
#p,d,q = 5,2,3
days = 26

def predictARIMA(X, p, d, q, days):
    model = ARIMA(X, order=(p,d,q))
    model_fit = model.fit()
    start = len(X)
    forecast = model_fit.predict(start = start, end = start + days)
    return(forecast)

def difference(dataset, interval = 1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i]-dataset[i-interval]
        diff.append(value)
    return np.array(diff)

# invert differenced value
def inverse_difference(history, yhat, interval=1):
    #yhat = np.exp(yhat)
    return yhat + history[-1]

from statsmodels.tsa.arima.model import ARIMA

proj = {}
for s in states:
    a = statesdata[s]
    a = a.reset_index()
    confirmed = a['Confirmed']
    deaths = a['Deaths']
    
    X = confirmed.values
    
    forecastC = predictARIMA(X, 2,2,1, days)
    # BEST PARAMS SO FAR ARE 2,2,1
    
    Y= deaths.values


    forecastD = predictARIMA(Y, 4,2,3, days)
    # BEST PARAMS SO FAR ARE 4,2,3

    df = {'Confirmed': forecastC, 'Deaths': forecastD}
    
    proj[s] = pd.DataFrame(df)

ModuleNotFoundError: No module named 'statsmodels'